In [1]:
import numpy as np
import pandas as pd
import requests
from PIL import Image
from sklearn.cluster import KMeans, MiniBatchKMeans

In [2]:
path = r'./Chungking-Express-14.jpg'

In [3]:
def online_image_loader(url):
    """
    Load an image from a given URL.

    Args:
        url: A string representing the URL of the image to be loaded.

    Returns:
        An Image object representing the loaded image.

    Raises:
        Exception: If the image could not be loaded from the given URL.

    Example:
        >>> image = online_image_loader('https://example.com/image.jpg')
    """
    response = requests.get(url, stream=True).raw
    try:
        image = Image.open(response)
    except:
        raise Exception('Image not found, try another URL')  
    return image

In [4]:
def local_image_loader(path):
    """
    Load an image from a given file path.

    Args:
        path: A string representing the file path of the image to be loaded.

    Returns:
        An Image object representing the loaded image.

    Raises:
        Exception: If the image could not be loaded from the given file path.

    Example:
        >>> image = local_image_loader('/path/to/image.jpg')
    """
    try:
        image = Image.open(path)
    except:
        raise Exception('Image not found, try another path')
    return image

In [5]:
def flatten_image(image):
    """
    Flatten an image into a pandas DataFrame.

    Args:
        image: An Image object representing the image to be flattened.

    Returns:
        A pandas DataFrame representing the flattened image. Each column in the
        DataFrame represents a color channel of the image.

    Example:
        >>> image = Image.open('/path/to/image.jpg')
        >>> flattened_image = flatten_image(image)
    """
    df = pd.DataFrame(list(image.getdata()),columns=list(image.getbands()))
    return df

In [6]:
img_local = local_image_loader(path)
# url = input('Enter image url: ')
# img_online = online_image_loader(url)

In [7]:
flat_local = flatten_image(img_local)
# flat_online = flatten_image(img_online)

In [8]:
class Clusterer():
    """
    A class for clustering RGB color values using different clustering algorithms.

    Attributes:
        df: A pandas DataFrame containing the RGB values of each color to be clustered.
        models: A dictionary storing the clustering models created by the class methods.

    Methods:
        KM: Create a KMeans clustering model and store it in the models dictionary.
        MBKM: Create a MiniBatchKMeans clustering model and store it in the models dictionary.

    Example:
        >>> clusterer = Clusterer(df)
        >>> km_model = clusterer.KM()
        >>> mbkm_model = clusterer.MBKM()
    """
    def __init__(self, df):
        """
        Initialize a new Clusterer object.

        Args:
            df: A pandas DataFrame containing the RGB values of each color to be clustered.
        """
        self.df = df
        self.models = {}

    def KM(self):
        """
        Create a KMeans clustering model and store it in the models dictionary.

        Returns:
            The KMeans clustering model.

        Example:
            >>> km_model = clusterer.KM()
        """
        self.models['KMeans'] = KMeans(n_clusters=8,n_init='auto').fit(self.df)
        return self.models['KMeans']
    
    def MBKM(self):
        """
        Create a MiniBatchKMeans clustering model and store it in the models dictionary.

        Returns:
            The MiniBatchKMeans clustering model.

        Example:
            >>> mbkm_model = clusterer.MBKM()
        """
        self.models['MiniBatchKMeans'] = MiniBatchKMeans(n_clusters=8,n_init='auto').fit(self.df)
        return self.models['MiniBatchKMeans']

In [29]:
def generate_rgb_palette(model,norm=False):
    """
    Generate a list of RGB color tuples based on the cluster centers of a given model.

    Args:
        model: A clustering model with a `cluster_centers_` attribute containing
            the RGB values of the cluster centers. This should be a NumPy array
            with shape (n_clusters, 3).
        norm (bool): Whether to normalize the RGB values to the range [0, 1] by
            dividing each value by 255. Default is False.

    Returns:
        A list of RGB color tuples. Each tuple has three integers between 0 and 255
        representing the red, green, and blue components of the color.

    Example:
        >>> from sklearn.cluster import KMeans
        >>> model = KMeans(n_clusters=5, random_state=0).fit(X)
        >>> generate_rgb_palette(model)
        [(255, 51, 0), (51, 153, 255), (255, 204, 0), (0, 102, 51), (255, 153, 204)]
    """
    if norm:
        palette = model.cluster_centers_ / 255.0
    palette = model.cluster_centers_.round().astype(int)
    palette_tuple = [tuple(p) for p in palette]
    return palette_tuple

def rgb_to_hex(rgb_value):
    """
    Convert an RGB color tuple to its hexadecimal representation.

    Args:
        rgb_value (tuple): A tuple containing three integers between 0 and 255
            representing the red, green, and blue components of a color.

    Returns:
        A string containing the hexadecimal representation of the color. The
        string will start with a '#' character followed by six hexadecimal digits
        representing the red, green, and blue components of the color.

    Example:
        >>> rgb_to_hex((255, 0, 0))
        '#ff0000'
        >>> rgb_to_hex((0, 128, 255))
        '#0080ff'
    """
    r, g, b = rgb_value[0], rgb_value[1], rgb_value[2]
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

def generate_hex_palette(model):
    """
    Generate a list of hexadecimal color codes based on the cluster centers of a given model.

    Args:
        model: A clustering model with a `cluster_centers_` attribute containing
            the RGB values of the cluster centers. This should be a NumPy array
            with shape (n_clusters, 3).

    Returns:
        A list of hexadecimal color codes. Each code is a string representing a
        6-digit hexadecimal number in the format "#RRGGBB", where RR, GG, and BB
        are two-digit hexadecimal numbers representing the red, green, and blue
        components of the color.

    Example:
        >>> from sklearn.cluster import KMeans
        >>> model = KMeans(n_clusters=5, random_state=0).fit(X)
        >>> generate_hex_palette(model)
        ['#ff3300', '#3399ff', '#ffcc00', '#006633', '#ff99cc']
    """
    palette = model.cluster_centers_.round().astype(int)
    palette = [rgb_to_hex(p) for p in palette]
    return palette

def hex_to_rgb(hex_value):
    """
    Convert a hexadecimal color code to an RGB color tuple.

    Args:
        hex_value: A string representing a 6-digit hexadecimal number in the
            format "#RRGGBB", where RR, GG, and BB are two-digit hexadecimal
            numbers representing the red, green, and blue components of the color.

    Returns:
        A tuple of three floats between 0 and 1 representing the red, green, and
        blue components of the color.

    Example:
        >>> hex_to_rgb('#ff3300')
        (1.0, 0.2, 0.0)
    """
    h = hex_value.lstrip('#')
    return tuple(int(h[i:i + 2], 16) / 255.0 for i in (0, 2, 4))

In [10]:
model = Clusterer(flat_local).KM()

In [30]:
rgb_palette = generate_rgb_palette(model)
hex_palette = generate_hex_palette(model)

In [33]:
for color in rgb_palette:
    print(color)

(21, 11, 5)
(115, 38, 3)
(97, 132, 2)
(154, 52, 3)
(203, 79, 4)
(80, 26, 2)
(66, 77, 1)
(47, 23, 2)


In [32]:
for color in hex_palette:
    print(color)

#150b05
#732603
#618402
#9a3403
#cb4f04
#501a02
#424d01
#2f1702
